In [24]:
from pymongo import MongoClient
import pandas as pd

In [25]:
client = MongoClient('mongodb://localhost:27017/')
db = client['flask_db']

In [26]:
def extract_user_info():
    collection = db['chat_client_info']
    data = list(collection.find({}, {'_id': 0, "session_id": 1, "client_id": 1, "product": 1, "grateful": 1, "ranting": 1, "expression": 1, "civil": 1}))
    df = pd.DataFrame(data)
    df.to_csv('user_info.csv', index=False)
    return df

def extract_chat_history():
    collection = db['chat_history_collection']
    data = list(collection.find({}, {'_id': 0, "session_id": 1, "client_id": 1, "turn_number": 1, "sender": 1, "receiver": 1, "message": 1, "timestamp": 1}))
    df = pd.DataFrame(data)
    df.to_csv('chat_history.csv', index=False)
    return df

def extract_user_feedback():
    collection = db['chat_emo_feedback']
    data = list(collection.find({}, {'_id': 0, "session_id": 1, "client_id": 1, "turn_number": 1, "support_type": 1, "support_content": 1, "timestamp_arrival": 1, "timestamp_feedback": 1, "user_feedback": 1}))
    df = pd.DataFrame(data)
    df.to_csv('user_feedback.csv', index=False)
    return df

In [27]:
# export csv run functions

if __name__ == "__main__":
    user_info_df = extract_user_info()
    chat_history_df = extract_chat_history()
    user_feedback_df = extract_user_feedback()

In [28]:
# Analyze the value from csv table

user_info_df = pd.read_csv('user_info.csv')
chat_history_df = pd.read_csv('chat_history.csv')
user_feedback_df = pd.read_csv('user_feedback.csv')


In [29]:
chat_history_df

,session_id,client_id,turn_number,sender,receiver,message,timestamp
0,72bb5300-a303-4e2d-bed9-48744a0e592c,f5b4a376-1aad-45c8-8e38-5e431c1de116,1,representative,client,There is no product mentioned in this complain...,2024-04-29 15:46:19.009
1,72bb5300-a303-4e2d-bed9-48744a0e592c,f5b4a376-1aad-45c8-8e38-5e431c1de116,1,client,representative,I feel so depressed,2024-04-29 17:21:58.572
2,72bb5300-a303-4e2d-bed9-48744a0e592c,f5b4a376-1aad-45c8-8e38-5e431c1de116,2,representative,client,"Hey, what's that got to do with me? I'm here b...",2024-04-29 17:21:58.572
3,72bb5300-a303-4e2d-bed9-48744a0e592c,f5b4a376-1aad-45c8-8e38-5e431c1de116,2,client,representative,I cannot help you,2024-04-29 17:23:45.679
4,72bb5300-a303-4e2d-bed9-48744a0e592c,f5b4a376-1aad-45c8-8e38-5e431c1de116,3,representative,client,I'm not asking for your inability to help; I'm...,2024-04-29 17:23:45.679
...,...,...,...,...,...,...,...
1150,2b6f3305-1854-4ebe-92a0-73f14c74b481,9b568f3e-4bfc-4cd6-b9c5-f66d03f4907c,3,representative,client,While I appreciate the offer of a discount for...,2024-06-27 16:56:05.408
1151,2b6f3305-1854-4ebe-92a0-73f14c74b481,9b568f3e-4bfc-4cd6-b9c5-f66d03f4907c,3,client,representative,I appreciate the quick response and action you...,2024-06-27 16:56:17.123
1152,2b6f3305-1854-4ebe-92a0-73f14c74b481,9b568f3e-4bfc-4cd6-b9c5-f66d03f4907c,4,representative,client,I'm glad to hear that you're committed to reso...,2024-06-27 16:56:17.123
1153,2b6f3305-1854-4ebe-92a0-73f14c74b481,9b568f3e-4bfc-4cd6-b9c5-f66d03f4907c,4,client,representative,I appreciate the quick response and action you...,2024-06-27 16:56:30.917


In [30]:
# User Info
print(user_info_df.describe())  # Basic statistics
print(user_info_df['product'].value_counts())
print(user_info_df['grateful'].mean())
print(user_info_df['ranting'].mean())
print(user_info_df['expression'].mean())

         grateful     ranting  expression       civil
count  344.000000  344.000000  344.000000  407.000000
mean     0.372093    0.636628    0.587209    0.538084
std      0.484067    0.481671    0.493053    0.499161
min      0.000000    0.000000    0.000000    0.000000
25%      0.000000    0.000000    0.000000    0.000000
50%      0.000000    1.000000    1.000000    1.000000
75%      1.000000    1.000000    1.000000    1.000000
max      1.000000    1.000000    1.000000    1.000000
product
hotel         135
Hotel          92
Airline        50
airline        42
Pizza          10
Book            9
Cup             8
Speaker         6
airplane        4
restaurant      1
airlines        1
Name: count, dtype: int64
0.37209302325581395
0.6366279069767442
0.5872093023255814


In [31]:
# Chat History
chat_history_df['timestamp'] = pd.to_datetime(chat_history_df['timestamp'])

# Report average message length for client and for rep
average_msg_length_client = chat_history_df[chat_history_df['sender'] == 'client']['message'].apply(len).mean()
average_msg_length_rep = chat_history_df[chat_history_df['sender'] == 'representative']['message'].apply(len).mean()
print("Average Message Length (Client):", average_msg_length_client)
print("Average Message Length (Rep):", average_msg_length_rep)

# Report the average time of response for client and for rep
response_times = chat_history_df.sort_values(by=['session_id', 'timestamp'])
response_times['response_time'] = response_times.groupby('session_id')['timestamp'].diff().dt.seconds
average_response_time_client = response_times[response_times['sender'] == 'client']['response_time'].mean()
average_response_time_rep = response_times[response_times['sender'] == 'representative']['response_time'].mean()
print("Average Response Time (Client):", average_response_time_client)
print("Average Response Time (Rep):", average_response_time_rep)

# Report the average number of turns
chat_history_df['turn_number'] = pd.to_numeric(chat_history_df['turn_number'], errors='coerce')  # Ensure turn numbers are numeric
max_turns_per_session = chat_history_df.groupby(['client_id', 'session_id'])['turn_number'].max().reset_index()
average_turns = max_turns_per_session.groupby('client_id')['turn_number'].mean()
print("Average Number of Turns per Session:", average_turns.mean())

Average Message Length (Client): 30.294444444444444
Average Message Length (Rep): 206.89685534591194
Average Response Time (Client): 229.0222222222222
Average Response Time (Rep): 418.80226904376013
Average Number of Turns per Session: 1.8275862068965518


In [32]:
# Analysis for User Feedback
average_feedback_by_type = user_feedback_df.groupby('support_type')['user_feedback'].mean()
print(average_feedback_by_type)

support_type
Be Mindful of Your Emotions           4.493506
Put Yourself in the Client's Shoes    4.482759
Sentiment Analysis                         NaN
You might be thinking                 4.395062
Name: user_feedback, dtype: float64


In [ ]:
# New testing！

In [33]:
print(chat_history_df['timestamp'].min())
print(chat_history_df['timestamp'].max())

2024-04-29 15:46:19.009000
2024-06-27 16:56:30.917000


In [34]:
conversation_duration = chat_history_df.groupby(['client_id', 'session_id']).agg({
    'timestamp': lambda x: (max(x) - min(x)).total_seconds() / 60
}).reset_index()
print(conversation_duration.describe())

        timestamp
count  435.000000
mean     4.100422
std     34.643486
min      0.000000
25%      0.000000
50%      0.000000
75%      0.204625
max    413.101017


In [35]:
negative_durations = conversation_duration[conversation_duration['timestamp'] < 0]
print(negative_durations)

Empty DataFrame
Columns: [client_id, session_id, timestamp]
Index: []


In [36]:
def clean_timestamps(group):
    return group.sort_values('timestamp')

chat_history_df = chat_history_df.groupby(['client_id', 'session_id']).apply(clean_timestamps).reset_index(drop=True)

/var/folders/_y/kw_r_gy137q8v8wn5psr60vh0000gn/T/ipykernel_59869/2861331544.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chat_history_df = chat_history_df.groupby(['client_id', 'session_id']).apply(clean_timestamps).reset_index(drop=True)


In [37]:
zero_duration = conversation_duration[conversation_duration['timestamp'] == 0]
print(f"Number of zero duration sessions: {len(zero_duration)}")

Number of zero duration sessions: 312


In [38]:
message_counts = chat_history_df.groupby(['client_id', 'session_id']).size().reset_index(name='message_count')
print(message_counts.describe())

       message_count
count     435.000000
mean        2.655172
std         4.204528
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max        25.000000


In [40]:
long_sessions = conversation_duration[conversation_duration['timestamp'] > 60]  # 超过1小时的会话
print(long_sessions)

                                client_id  \
100  3d5398df-3fb0-4b56-9a15-3008e6a15047   
227  8d3e7706-4270-48ab-9983-378c708d47c5   
283  af607d51-a31b-4d26-9cfd-3676eda58529   
411  f35aec4f-8521-4d2f-a2a8-29efe9b584f6   
415  f5b4a376-1aad-45c8-8e38-5e431c1de116   

                               session_id   timestamp  
100  451a2c20-4d89-4814-9749-5e28e086a7ef  406.207317  
227  451a2c20-4d89-4814-9749-5e28e086a7ef  413.101017  
283  ffe49cdc-9d18-4c1a-8d0c-5cec89ad1028   72.001267  
411  82da3f09-d376-4020-a401-aecf7115d61b  411.033933  
415  72bb5300-a303-4e2d-bed9-48744a0e592c   97.444500  


In [51]:
# How long does a conversation with a client last. Total length + Total time + Total turns.
conversation_metrics = chat_history_df.groupby(['client_id', 'session_id']).agg({
    'message': lambda x: sum(len(msg) for msg in x),
    'timestamp': lambda x: (max(x) - min(x)).total_seconds() / 60,
    'turn_number': 'max'
}).reset_index()

conversation_metrics.columns = ['client_id', 'session_id', 'total_length', 'total_time_minutes', 'total_turns']

# the average of time and turns across clients （Total length + Total time + Total turns.）
avg_length = conversation_metrics['total_length'].mean()
avg_time = conversation_metrics['total_time_minutes'].mean()
avg_turns = conversation_metrics['total_turns'].mean()

print("-"*30)
print(f"Overall average conversation length: {avg_length:.2f} characters")
print(f"Overall average conversation time: {avg_time:.2f} minutes")
print(f"Overall average conversation turns: {avg_turns:.2f}")

# call user_info_df to analyze based on emo and civil
merged_df = conversation_metrics.merge(user_info_df, on=['client_id', 'session_id'])

#  the difference in length, time and turns when emo=1 vs emo=0
emo_1 = merged_df[merged_df['grateful'] == 1]
emo_0 = merged_df[merged_df['grateful'] == 0]

print("-"*30)
print("\nAverages when emo=1:")
print(f"Length: {emo_1['total_length'].mean():.2f} characters")
print(f"Time: {emo_1['total_time_minutes'].mean():.2f} minutes")
print(f"Turns: {emo_1['total_turns'].mean():.2f}")

print("\nAverages when emo=0:")
print(f"Length: {emo_0['total_length'].mean():.2f} characters")
print(f"Time: {emo_0['total_time_minutes'].mean():.2f} minutes")
print(f"Turns: {emo_0['total_turns'].mean():.2f}")

#  the difference in length, time and turns when civil=1 vs civil=0
civil_1 = merged_df[merged_df['civil'] == 1]
civil_0 = merged_df[merged_df['civil'] == 0]

print("-"*30)
print("\nAverages when civil=1:")
print(f"Length: {civil_1['total_length'].mean():.2f} characters")
print(f"Time: {civil_1['total_time_minutes'].mean():.2f} minutes")
print(f"Turns: {civil_1['total_turns'].mean():.2f}")

print("\nAverages when civil=0:")
print(f"Length: {civil_0['total_length'].mean():.2f} characters")
print(f"Time: {civil_0['total_time_minutes'].mean():.2f} minutes")
print(f"Turns: {civil_0['total_turns'].mean():.2f}")

print("-"*30)
# the time to complete post-task survey
user_feedback_df['timestamp_arrival'] = pd.to_datetime(user_feedback_df['timestamp_arrival'])
user_feedback_df['timestamp_feedback'] = pd.to_datetime(user_feedback_df['timestamp_feedback'])
user_feedback_df['survey_completion_time'] = (user_feedback_df['timestamp_feedback'] - user_feedback_df['timestamp_arrival']).dt.total_seconds() / 60
avg_survey_time = user_feedback_df['survey_completion_time'].mean()
print(f"\nAverage time to complete post-task survey: {avg_survey_time:.2f} minutes")


------------------------------
Overall average conversation length: 403.19 characters
Overall average conversation time: 4.10 minutes
Overall average conversation turns: 1.83
------------------------------

Averages when emo=1:
Length: 421.34 characters
Time: 1.28 minutes
Turns: 1.89

Averages when emo=0:
Length: 356.20 characters
Time: 4.40 minutes
Turns: 1.80
------------------------------

Averages when civil=1:
Length: 391.69 characters
Time: 2.70 minutes
Turns: 1.72

Averages when civil=0:
Length: 398.49 characters
Time: 5.09 minutes
Turns: 1.91
------------------------------

Average time to complete post-task survey: 6.46 minutes
